<img src="img/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

# Exploración de una imagen

**Introducción**

El Open Data Cube es un conjunto de librerías **Python** que facilitan el proceso de **Organización**, **Consulta** y **Recuperación** de información de imágenes de satélite. 

Estos análisis se realizan mediante comandos en el lenguaje Python que usan precisamente las funciones provistas por las librerías que hemos nombrado.

Este notebook muestra el proceso de consulta y exploración y visualización básico de **UNA** imagen satelital mediante las funcionalidades que ofrece el ODC. En consecuencia, las instrucciones utilizadas trabajan bajo la premisa que hay una sola imagen em el cubo. En notebooks posteriores se trabaja ya con más de una imagen y aunque se utilicen las mismas funciones, se deben utilizar con parámetros diferentes.

Los fundamentos de Python necesarios se encuentran en el notebook `1. Introducción a Python.ipynb` en el directorio notebooks base: [Aquí](http://localhost:8080/lab/tree/notebooks%20base/1.%20Introducci%C3%B3n%20a%20Python.ipynb) 

**Objetivo**

El objetivo del presente notebook es familiarizarse con proceso de consulta de imágenes satelitales en el cubo de datos, a conocer la estructura de matrices en la cuál el cubo de datos entrega los datos de imágenes satelitales y finalmente familiarizarse con el proceso de visualización de imágenes. 

**Contenido**

1. Instalar librerías de Python
2. Importar librerías
3. Especificación del área de estudio
4. Búsqueda y visualización de imágenes en el Open Data Cube
5. Exploración de la estructura de datos de una imagen en el cubo
6. Visualización de las bandas individuales

## 1. Instalar librerías de Python
___

Dado que el cubo se trabaja desde Python, se puede utilzar cualquiera de las librerías de Python ya existentes.

En esta sección se inatala la librería `geopandas`, que permite, en particular, usar polígonos en formato KML (el de Google Earth) en Python. Esta instalación es necesaria únicamente la primera vez que se ejecuta este notebook.

Para instalar una librería se usa el comando 

`pip3 install <nombre librería de python>`


In [ ]:
!pip3 install geopandas

**Una vez la libería es instalada es necesario reiniciar el Kernel de Jupyter. Para ello debe ir a la `barra de heramientas -> Kernel -> Restart Kernel...`. Luego en la opción `Restart`.**

## 2. Importar librerías
___
En esta sección se importan las librerías cuya funcionalidades particulares son requeridas.

**Nota**: Si le sale un error que dice que no reconoce geopandas es porque no ha reinicializado el kernel, como se indica en la sección anterior

In [ ]:
# las funcionalidades del open data cube son accedidas 
# por medio de la librería datacube
import datacube

# Librería usada para operaciones matemáticas
import numpy as np

# Librería usada para visualización de datos
import matplotlib.pyplot as plt

# Desactiva los warnings en el notebook
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Librería usada para la carga de polígonos
import geopandas as gpd

# Configuración de Drivers para leer polígonos en formato KML
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

## 3. Especificación del área de estudio
Para usar el cubo de datos y las imágenes que contiene, lo primero que hay que hacer es contarle al cubo cuál es el área en la que se quiere trabajar, expresada como un rectángulo que define sus límites.

En este notebook, esta área de estudio se especifica a partir de un punto.

Los coordenadas del punto a seleccionar pueden ser obtenidas a través de herramientas GIS como Google Maps. Este punto debe estar comprendido en el área que desea estudiar. El punto definido se emplea para la generación de un cuadrado que finalmente se usa para consultar el área de estudio. La variable `buffer` permite ampliar o disminuir las dimensiones de dicho cuadrado. Lo anterior es equivalente a disminuir o ampliar el área de estudio a consultar en el open data cube.

<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/latlong_buffer.png" alt="Definición área de estudio" width="20%">

In [ ]:
# Definición de las coordenadas del punto central del área de estudio
central_lat = 5.547964746532565
central_lon =  -72.9284962779124

# Aumento del aŕea del cuadrado para "EPSG:4326" (WGS 84)
# 11.1 kilómetros
buffer = 0.1

# Cálculo del cuadro delimitador (bounding box) para el área de estudio
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

print(f'    latitude={study_area_lat},')
print(f'    longitude={study_area_lon},')

## 4. Búsqueda y visualización de imágenes en el Open Data Cube
___
El siguiente paso es encontrar entre todas las imágenes que tiene el cubo, aquellas que cubren el área de estudio y son de interés para el análisis que se quiere desarrollar. 

Para eso el cubo de datos provee la función *load*, en la que por medio de parámetros se le indica cuáles son las imágenes de interés, siendo las más usuales el área de estudio y el período de tiempo que se quiere estudiar. Toda la información solicitada es extraida de las imágenes que cumplen las condiciones de búsqueda y cargadas (de ahí el nombre de la función) en una estructura de datos (*xarray*) apta para su procesamiento.

Para la visualización, se toman tres bandas de la imagen y se especifica el color con el cual se va a dibujar: la primera se dibuja en rojo, la segunda en verde y la tercera en azul. Luego de una manipulación para acomodar la información, se utiliza la función imshow.

La siguiente celda realiza la búsqueda de la única imagen que hay en el cubo por el momento y la visualiza en *verdadero color* escogiendo las bandas roja, verde y azul y dibujándolas con su respectivo color.

**Nota**: Los parámetros vmin y vmax especifican los valores mínimo y máximo de visualización de la información y determinan la escala de colores que se utiliza para la visualización: todos los valores menores a vmin se pasan a vmin y todos los valores mayores a vmax se pasan a vmax. Para esta imagen es necesario especificarlos, pues hay algunos pixeles fuera de rango (pueden verificarlo quitando los parámetros y volviendo a ejecutar la celda)

Para mayor información los enlaces a la documentación oficial son:

- https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html
- http://xarray.pydata.org/en/stable/user-guide/index.html
- http://xarray.pydata.org/en/stable/generated/xarray.plot.imshow.html

In [ ]:
dc = datacube.Datacube(app="MOOC GEO")

dataset = dc.load(
    product="s2_sen2cor_ard_granule_EO3",                   # El sensor - Sentinel 2
    latitude=study_area_lat,                                # Los límites en latitud del área de estudio
    longitude=study_area_lon,                               # Los límites en longitud del área de estudio
    time=('2021-01-01', '2021-02-01'),                      # El período de tiempo de interés
    measurements=["red","blue","green","nir","swir1","swir2","scl"],  # Las bandas que se quieren estudiar
    crs="EPSG:4326",                                        # El sistema de coordenadas de entrada (WGS 84)
    output_crs="EPSG:4326",                                 # El sistema de coordenadas de la respuesta (WGS 84)
    resolution=(-0.00008983111,0.00008971023)               # Precisión de la respuesta, en grados
)

# Dibujar la imagen en color verdadero (`true color`)
rgb = dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb[0].plot.imshow(vmin=0, vmax=1200,        # rango de valores visibles
                         size=8, aspect = 1)      # tamaño y proporción de la visualización

Recordemos que también se puede visualizar las imágenes en *falso color*, con el objetivo de resaltar algunas características de la zona de estudio, no visibles al ojo humano.

Una de las más utilizadas es el *falso color* para resaltar la vegetación, que visualiza la banda del nir (infrarojo cercano) en rojo, la banda del rojo en verde y la banda el verde en azul.


Mayor información y otras combinaciones interesantes en:
- https://acolita.com/lista-de-combinaciones-de-bandas-en-sentinel-2a/

In [ ]:
# Definimos una nueva variable rgb2 de manera similar a como se definió la variable rgb en la celda anterior.
rgb2 = dataset[["nir","red","green"]].to_array(dim='color')
rgb2 = rgb2.transpose(*(rgb2.dims[1:]+rgb2.dims[:1]))  # make 'color' the last dimension

# Definimos una nueva variable img2, basada en rgb2
img2 = rgb2[0].plot.imshow(vmin=0, vmax=1200, size=10, aspect = 1)

> **Ejercicio**

> Replicar, con base en las celdas anteriores, la operación de visualizar la información de imágenes satelitales.

> Escoger por lo menos dos de las presentadas en https://acolita.com/lista-de-combinaciones-de-bandas-en-sentinel-2a/

> Definir nuevas variables rgbX e imgX para cada caso

## 5. Exploración de la estructura de datos de una imagen en el cubo
___
La visualización de las imágenes es interesante, pero para analizarlas, es necesario conocer la estructura de datos interna.

La función `dc.load` del open datacube recibe una consulta y retorna una estructura de datos llamada `xarray.Dataset` que contiene toda la información que fue solicitada en la consulta.

<img src="https://raw.githubusercontent.com/MINE-Coursera/GEO_M4_DataCube/Desarrollo/notebooks%20ODC/img/xarray.png" alt="xarray.Dataset" width="60%">

**Nota:** La imagen mostrada es genérica, que representa un conjunto de datos n-dimensional. Las variables de datos son la temperatura y la precipitación. Para el caso específico de las imágenes de satélite, ahí están las bandas espectrales de la imagen (rojo, verde, azul, infrarojo, ...).

Y para el ejercicio actual, en que solo tenemos una imagen, solo tendríamos un plano para cada una de las variables de datos.

### La estructura de un xarray
Un `xarray.Dataset` tiene la siguiente estructura:

**Dimensiones (`Dimensions`)**

Las dimensiones determinan el dominio espaciotemporal de los datos. Identifican el número de instantes de tiempo devueltos en la búsqueda, así como el número de valores en las dimensiones `latitude` y `longitude`. Además, indica cómo están organizados los datos en las diferentes dimensiones.

**Coordenadas (`Coordinates`)**

* `time` identifica la fecha atribuida a cada intervalo de tiempo devuelto como resultado de la consulta.
* `latitude` y `longitude` son los valores de las coordenadas que dentro de los límites espaciales de la consulta.

**Variables de datos (`Data variables`)**

* Cada banda (*measurement*) que representa la información espectral de la imagen se representa en una *variable de datos*. 
* En el ejemplo anterior, las variables de datos obtenidas son: `red`,`blue`,`green`,`nir`,`swir1`,`swir2`,`scl`. 
* Cada variable de datos es un arreglo de tres dimensiones de tipo `xarray.DataArray`. Las tres dimensiones son  (`time, latitude, longitude`)

<img src="https://raw.githubusercontent.com/MINE-Coursera/GEO_M4_DataCube/Desarrollo/notebooks%20ODC/img/numpy_array_shape.png" alt="xarray.Dataset" width="60%">


**Atributos (`Attributes`)**

* `crs` identifica el sistema de coordenadas de referencia de la imagen obtenida.


In [ ]:
# Para ver la imagen, en su estructura de datos, simplemente se le pide a Python que la escriba
dataset

El dataset está estructurado como se indicó en la celda anterior.

Explore los valores individuales con los botones de *hoja* y *disco* a la derecha de los campos.
- El *disco* muestra los valores contenidos en la estructura de datos. Para los arreglos, muestra los tres primeros y los tres últimos valores del arreglo y puntos suspensivos en la mitad.
- La *hoja* muestra los metadatos (información adicional necesaria para interpretar correctamente los datos), como las unidades, el sistema de coordenadas, ...

Como no siempre se quiere o es útil trabajar con toda la estructura de datos al tiempo, es posible extraer y trabajar con los elementos indivuduales.

Las siguientes celdas muestran cómo hacerlo

### Exploración de las dimensiones

La propiedad `dims` permite visualizar las dimensiones del dataset.

In [ ]:
dataset.dims

### Exploración de las coordenadas

La propiedad `coords`permite visualizar las coordenadas. Las coordenadas pueden ser vistas como las etiquetas de los ejes de un cubo de tres dimensiones. En este caso, las coordenadas son `time`, `longitude`, `latitude`.

Para la correcta interpretación está especificada el sistema de coordenadas 4326 - WGS 84

In [ ]:
dataset.coords

### Exploración de las fechas de las imágenes

Para conocer de forma explícita las fechas en que fueron tomadas las imágenes resultado de la consulta. 

Recordemos que la primera vez que se ejecuta este notebook solo hay una imagen en el cubo. Sin embargo, verificamos y guardamos la información del número de imágenes. Esto lo necesitamos para la posterior visualización de las bandas individuales.

In [ ]:
dataset.coords['time']

In [ ]:
num_imagenes = dataset.coords ['time'].shape [0]
num_imagenes

### Exploración de las coordenadas de longitud

Para conocer de forma explícita las coordenadas de longitud

In [ ]:
dataset.coords['longitude']

### Exploración de las coordenadas de latitud

Para conocer de forma explícita las coordenadas de latitud

In [ ]:
dataset.coords['latitude']

### Exploración de las variables de datos

La información espectral de una imagen satelital está organizada por el Open Data Cube en *variables de datos*. 

Cada variable de datos contiene la información de una única banda. 

La información de cada banda es organizada en un arreglo de tres dimensiones. 

Para acceder a la información espectral de la banda `blue` se puede usar la expresión mostrada a continuación.

In [ ]:
dataset.blue

Dado que cada variable de datos es un arreglo de tres dimensiones, es posible indexar la información del mismo. 

Es importante conocer el orden de los ejes para determinar de antemano la información que se muestra al indexar una variable de datos: la primera dimensión es el tiempo, la segunda es la latitud y la tercera es la longitud.

Por ejemplo, puedo obtener la primera imagen en el tiempo para la banda `blue`. 

**Nota:** En este caso la respuesta es igual a la anterior consulta, pues sólo tenemos una imagen en el cubo

In [ ]:
dataset.blue[0]

De la misma forma, se puede ver el valor para la banda `blue` de un único píxel. Los valores entre corchetes `[0,0,0]` se interpretan considerando el orden de las coordenadas. En este caso:

* la primera coordenada es el tiempo (`time`) de modo que en la matrix se ha seleccionado el periodo de tiempo `0`. 
* la segunda coodenada es la latitud (`latitude`) de modo que en la matrix se ha seleccionado la latitud `0`
* la tercera coordenada es la longitud (`longitude`) así, la longitud seleccionada es `0`

In [ ]:
dataset.blue[0,0,0]

El ejemplo anterior depende de la definición del área de estudio, que es la que define la posición 0 de los arreglos. Esto no es my práctico y también, generalmente el analista puede estar interesado en un punto específico, definido por sus coordenadas geográficas.

La siguiente celda muestra cómo se puede el valor de la banda azul, en la imagen más antigua indexada en el cubo, en unas coordenadas específicas `(5.63901318, -73.01502163)`, representadas en las variables lat_objetivo y long_objetivo.

Y también ilustra el problema y el manejo de números reales en un computador: la precisión, que se maneja mediante un rango de valores. Este rango es muy peqaueño y está definido por la variable `epsilon`.

In [ ]:
epsilon = 0.00000001               # La precisión de la comparación
lat_objetivo = 5.63901318          # La latidud del sitio de interés
long_objetivo = -73.01502163       # La longitud del sitio de interés

# Renombramiento de las dimensiones `latitude` y `longitude`, para mejorar la legibilidad del código
lat = dataset.coords ['latitude']
long = dataset.coords ['longitude']

# Cálculo de la posición (índice) de la coordenada de latitud buscada en la dimensión `latitude`
pos_lat = np.where ((lat >= lat_objetivo - epsilon) & (lat <= lat_objetivo + epsilon))

# Cálculo de la posición (índice) de la coordenada de longtud buscada en la dimensión `longitude`
pos_long = np.where ((long >= long_objetivo - epsilon) & (long <= long_objetivo + epsilon))

# La función `whre` devuelve un arreglo con todas las posiciones donde se encuentra el valor buscado. 
# Sabiendo que las dimensiones de `latitude` y `longitude` están ordenados y no tienen elementos repetidos, 
# nos quedamos con la primera (y única posición)

dataset.blue [0, pos_lat[0], pos_long [0]]

> **Ejercicio:**  Realice el proceso de **Exploración de variables de datos** para las variables (bandas) restantes `red`,`green`,`nir`,`swir1`,`swir2` y `scl`. Cree nuevas celdas para cada caso, siguiendo el ejemplo de la banda `blue`.

### Exploración de atributos

Entre los atributos que hacen parte de los metadatos de la imagen, se muestra el sistema de referencia de coordenadas (CRS) de la imagen obtenida a partir del Open Data Cube. 

Una de las grandes ventajas del Open Data Cube es que permite obtener la información de las imágenes en diferentes sistemas de coordenadas y en diferentes dimensiones. En el momento de solicitar la información, el cubo realiza las operaciones y proyecciones necesarias para entregar la información como la necesita el analista

In [ ]:
dataset.crs

## 6. Visualización de las bandas individuales
___
Para visualizar una banda particular se utiliza el comando `plot` a la variable de datos deseada, utilizando la paleta de colores por defecto.

Acá queremos analizar la primera imagen (la más antigua). El código para esto depende del número de imágens que hayan sido indexadas en el cubo. La primera vez que se ejecute este notebook, se espera que haya solo una imagen, pero para posteriores ejecuciones es posible que haya más de una.

In [ ]:
if num_imagenes == 0:
    print ('No hay imágenes en el cubo')
elif num_imagenes == 1:
    dataset.blue.plot(vmin=0,vmax=1200, size=10, aspect=1)
else:
    dataset.blue[0].plot(vmin=0,vmax=1200, size=10, aspect=1)

Para cambiar el rango de colores de la visualización mostrada se usa el parámetro `cmap` que indica la paleta de colores que se quiere utilizar.

El listado de paletas de colores válids está [aquí](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
if num_imagenes == 0:
    print ('No hay imágenes en el cubo')
elif num_imagenes == 1:
    dataset.blue.plot(vmin=0,vmax=1200, cmap='Blues', size=10, aspect=1)
else:
    dataset.blue[0].plot(vmin=0,vmax=1200, cmap='Blues', size=10, aspect=1)

In [ ]:
if num_imagenes == 0:
    print ('No hay imágenes en el cubo')
elif num_imagenes == 1:
    dataset.red.plot(vmin=0,vmax=1200, cmap='Reds', size=10, aspect=1)
else:
    dataset.red[0].plot(vmin=0,vmax=1200, cmap='Reds', size=10, aspect=1)

> **Ejercicio:**  Realice el proceso de **Visualización** para las variables (bandas) restantes `green`,`nir`,`swir1`,`swir2` y `scl`. Cree una nueva celda para cada ejemplo. Puede apoyarse en el ejemplo de exploración de la banda `blue` mostrado anteriormente. Use los colores `'Greens'` para la banda `green`; para las demás bandas no cambie los colores.